In [1]:
%reload_ext autoreload
%autoreload 2

In [83]:
import radix
from nbc_analysis.utils.toml_utils import get_config
from nbc_analysis.utils.file_utils import read_parquet

from pathlib import Path
from toolz import take

In [30]:
config = get_config('test')
normalize_cfg = config['normalize']
demographics_cfg = config['demographics']
config

{'demographics': {'demographics_d': '/Users/wmcabee/DATA/NBC/demographics',
  'zip2income_input_f': PosixPath('/Users/wmcabee/_NBC/datasets/ACS_17_5YR_S2503_with_ann.csv'),
  'subnet2zip_input_f': PosixPath('/Users/wmcabee/_NBC/datasets/GeoLite2-City-CSV_20191001.zip'),
  'subnet2inc_filename': 'subnet2inc'},
 'normalize': {'test_input_d': '/Users/wmcabee/DATA/NBC/work/query',
  'normalize_d': '/Users/wmcabee/DATA/NBC/normalize'}}

In [42]:
indir = Path(normalize_cfg['normalize_d'])
IPS = read_parquet('ips',indir=indir)
IPS

2019-10-22 07:50:08,737|INFO|nbc_analysis.utils.file_utils|start read file,/Users/wmcabee/DATA/NBC/normalize/ips.parquet
2019-10-22 07:50:08,809|INFO|nbc_analysis.utils.file_utils|end read file,/Users/wmcabee/DATA/NBC/normalize/ips.parquet,record_cnt=226596


,ip
0,1.144.107.183
1,1.144.110.127
2,1.144.110.252
3,1.152.111.20
4,1.159.207.45
...,...
226591,99.99.221.220
226592,99.99.240.240
226593,99.99.25.135
226594,99.99.26.73


In [46]:
indir = Path(demographics_cfg['demographics_d'])
SUBNET2INC = read_parquet('subnet2inc.parquet', indir=indir, columns=['network', 'network_key'])
SUBNET2INC.iloc[:10]

2019-10-22 07:55:36,875|INFO|nbc_analysis.utils.file_utils|start read file,/Users/wmcabee/DATA/NBC/demographics/subnet2inc.parquet
2019-10-22 07:55:37,513|INFO|nbc_analysis.utils.file_utils|end read file,/Users/wmcabee/DATA/NBC/demographics/subnet2inc.parquet,record_cnt=1243176


,network,network_key
0,2.56.9.245/32,340298234694711661
1,2.56.9.246/32,15967697370209441848
2,2.56.9.247/32,8344275163890935531
3,2.56.9.248/32,17455670536690040679
4,2.56.20.0/22,3447722192060392118
5,2.56.32.0/22,10079855123282555649
6,2.56.44.0/22,5620104867241794362
7,2.56.48.0/22,11967419260617987618
8,2.56.56.0/22,16582982096803622195
9,2.56.114.0/23,12182769773387179504


In [51]:
%%time
RTREE = radix.Radix()

def add_network(rec):
    RTREE.add(rec.network).data['network_key'] = rec.network_key

reader = subnet2inc.itertuples()
#reader = take(100, reader)
reader = map(add_network, reader)
for x in reader: pass

CPU times: user 2min 36s, sys: 5.57 s, total: 2min 41s
Wall time: 2min 42s


In [58]:
IPS.sample(10)

,ip
11917,13.233.52.178
21366,173.152.23.88
88073,2600:387:0:9c2::9
196418,71.183.83.246
123887,2601:603:1400:b72:dc0a:42a6:5033:3cd5
195978,71.163.14.137
37194,204.42.19.168
29671,174.84.250.36
76533,2600:1700:f790:15b0:6986:73f2:93a6:d6cd
88154,2600:387:1:803::94


In [81]:
%%time
def get_network_key(ip):
    rnode = RTREE.search_best(ip)
    return 0 if rnode is None else rnode.data['network_key']

#N = 100000
#df = IPS.sample(N)
df = IPS
df['network_key'] = df.ip.map(get_network_key)
(df.network_key != 0).sum() / N

CPU times: user 26.7 s, sys: 868 ms, total: 27.6 s
Wall time: 27.6 s


2.09712

In [ ]:
RTREE.